In [2]:
import duckdb
import pandas as pd
import os

# Construct the path to your database relative to the notebook location
DB_PATH = os.path.join("..", "data", "simulation.db")

# Connect to the DuckDB database
conn = duckdb.connect(DB_PATH)

In [ ]:
# display orders table
df = conn.execute("SELECT * FROM order_events").fetchdf()
df.head(5)

In [ ]:
cycle_time_query = """
SELECT
    AVG(end_pick_time - arrival_time) as time_in_system
FROM order_events
WHERE end_pick_time IS NOT NULL 
"""

workload_query = """
SELECT
    a.picker_id,
    c.shift_name,
    d.skill_name,
    COUNT(a.order_id) AS picked_orders
FROM 
    order_events a
    INNER JOIN pickers b ON a.picker_id = b.picker_id
    INNER JOIN shifts c ON b.shift_id = c.shift_id
    INNER JOIN skills d ON b.skill_id = d.skill_id
WHERE a.picker_id IS NOT NULL
GROUP BY a.picker_id, c.shift_name, d.skill_name
ORDER BY COUNT(a.order_id) DESC
"""


In [ ]:
conn.execute(cycle_time_query).fetchdf()

In [ ]:
conn.execute(workload_query).fetchdf()


In [ ]:
conn.execute("SELECT * FROM shifts").fetchdf()

In [ ]:
conn.execute("SELECT * FROM skills").fetchdf()

In [ ]:
conn.execute("SELECT * FROM picker_states LIMIT 20").fetchdf()

In [ ]:
conn.execute("""
SELECT
    picker_id,
    state,
    timestamp AS start_time,
    LEAD(timestamp) OVER (PARTITION BY picker_id ORDER BY timestamp) AS end_time
FROM picker_states
ORDER BY picker_id, timestamp;
""").fetch_df()

In [ ]:
conn.execute("""
    WITH states AS (
        SELECT
            picker_id,
            state,
            timestamp AS start_time,
            LEAD(timestamp) OVER (PARTITION BY picker_id ORDER BY timestamp) AS end_time,
            LEAD(timestamp) OVER (PARTITION BY picker_id ORDER BY timestamp) - timestamp AS duration
        FROM picker_states
        ),
    states_duration AS (                      
        SELECT
            picker_id,
            state,
            SUM(duration) as state_duration
        FROM states
        GROUP BY picker_id, state
        )
    SELECT 
        picker_id,
        state,
        state_duration,
        SUM(state_duration) OVER (PARTITION BY picker_id) AS total_duration,
        ROUND((state_duration / SUM(state_duration) OVER (PARTITION BY picker_id)) * 100, 2) AS prop     
    FROM states_duration
""").fetch_df()

In [ ]:
conn.execute("""
WITH events AS (
    SELECT arrival_time AS t, +1 AS delta FROM order_events
    UNION ALL
    SELECT end_pick_time AS t, -1 AS delta FROM order_events
)
SELECT
    t,
    SUM(delta) OVER (ORDER BY t ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS orders_in_system
FROM events
WHERE t IS NOT NULL
ORDER BY t
""").fetch_df()

In [ ]:
conn.execute("""
    SELECT
        SUM(CASE WHEN end_pick_time <= due_date THEN 1 ELSE 0 END) * 1.0 / COUNT(order_id) AS prop_on_time
    FROM order_events
    WHERE end_pick_time IS NOT NULL
""").fetch_df()

In [6]:
conn.close()